In [1]:
#homemade script
from util import Gini
from feature_generater import Multiply_Divide, Series_string, Features_Counts, Statistic_features

import numpy as np
import pandas as pd
import pickle
from scipy import sparse
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

#for NN model
from keras.layers import Dense, Dropout, Embedding, Flatten, Input, Concatenate, merge
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.models import Model
from time import time
import datetime
from sklearn.model_selection import StratifiedKFold


/Users/stevenhu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# 1. Load Data #

- Create train, test dataset
- Create train target label
- Create feature object: cat, num, bin, inter
- Create feature columns in train: counting of miss values


In [2]:
cv_only = True
save_cv = True

#read data
train = pd.read_csv("../input/train.csv")
train_label = train['target']
train_id = train['id']
del train['target'], train['id']

test = pd.read_csv("../input/test.csv")
test_id = test['id']
del test['id']



#find missing value by each row and recode to column 'missing'
train['missing'] = (train==-1).sum(axis=1).astype(float)
test['missing'] = (test==-1).sum(axis=1).astype(float)

#get all featrue name
feature_names = list(train)

# extract feature with cat, bin, num, inter
cat_fea = [x for x in list(train) if 'cat' in x]
bin_fea = [x for x in list(train) if 'bin' in x]
num_features = [c for c in list(train) if ('cat' not in c and 'calc' not in c)]
inter_fea = [x for x in list(train) if 'inter' in x]

# 2. Feature Engineering #

- Create Multipy and Divide feature
- Feature of Counts(target incoding)
- Load feature generated from Feature Engine
- Create Statistic features
- Combine all feature together, and get ready for training
- Create Cat_feature for NN embeding training

## 2.1 Multipy and Divide feature ##
- moltipy each feature in the list and created new columns into train and testing data set

In [3]:
#Add features of Multiply and Divide
features= ['ps_car_13', 'ps_ind_03', 'ps_reg_03', 'ps_ind_15', 'ps_reg_01', 'ps_ind_01']
train, test, MD_features = Multiply_Divide(train, test, features)


In [4]:
display(train[MD_features].head(5))

,inter_0*,inter_0/,inter_1*,inter_1/,inter_2*,inter_2/,inter_3*,inter_3/,inter_4*,inter_4/,...,inter_10*,inter_10/,inter_11*,inter_11/,inter_12*,inter_12/,inter_13*,inter_13/,inter_14*,inter_14/
0,4.418395,0.176736,0.634544,1.230630,9.720468,0.080334,0.618575,1.262398,1.767358,0.441839,...,0.502649,1.025815,1.436141,0.359035,7.7,15.714286,22,5.500000,1.4,0.350000
1,4.331716,0.088402,0.474062,0.807773,1.856450,0.206272,0.495053,0.773521,0.618817,0.618817,...,0.612862,0.957597,0.766078,0.766078,2.4,3.750000,3,3.000000,0.8,0.800000
2,5.774271,0.071287,-0.641586,-0.641586,7.699029,0.053465,0.000000,inf,3.207929,0.128317,...,-0.000000,-inf,-5.000000,-0.200000,0.0,inf,60,2.400000,0.0,0.000000
3,1.085898,0.271474,0.315425,0.934592,4.343590,0.067869,0.488654,0.603276,0.000000,inf,...,0.522853,0.645497,0.000000,inf,7.2,8.888889,0,inf,0.0,inf
4,0.000000,inf,0.475728,0.673001,5.092484,0.062870,0.396082,0.808331,0.000000,inf,...,0.588531,1.201084,0.000000,inf,6.3,12.857143,0,inf,0.0,inf


## 2.2 Feature of Counts ##
1. Generate new_ind, new_reg, new_car
2. Count the number of distinct values of
    cat features, new_ind, new_reg and new_car

In [4]:

'''
create 1_0_1_1..... data as new_xxx

new_ind: collect all data from all relative "ind" columns, then generate series number

new_reg, new_car for train and test data 
For RNN processing, generating a sequence number
'''


category_list = ['ind', 'reg', 'car']
#add 'new_ind','new_reg','new_car' in train and test dataset
train, test = Series_string(train,test,category_list )



In [6]:
display(train[['new_ind','new_reg','new_car']].head(5))

,new_ind,new_reg,new_car
0,2_2_5_1_0_0_1_0_0_0_0_0_0_0_11_0_1_0,0.7_0.2_0.7180703308,10_1_-1_0_1_4_1_0_0_1_12_2_0.4_0.8836789178_0....
1,1_1_7_0_0_0_0_1_0_0_0_0_0_0_3_0_0_1,0.8_0.4_0.7660776723,11_1_-1_0_-1_11_1_1_2_1_19_3_0.316227766_0.618...
2,5_4_9_1_0_0_0_1_0_0_0_0_0_0_12_1_0_0,0.0_0.0_-1.0,7_1_-1_0_-1_14_1_1_2_1_60_1_0.316227766_0.6415...
3,0_1_2_0_0_1_0_0_0_0_0_0_0_0_8_1_0_0,0.9_0.2_0.5809475019,7_1_0_0_1_11_1_1_3_1_104_1_0.3741657387_0.5429...
4,0_2_0_1_0_1_0_0_0_0_0_0_0_0_9_1_0_0,0.7_0.6_0.840758586,11_1_-1_0_-1_14_1_1_2_1_82_3_0.3160696126_0.56...


In [5]:
'''
count_features

preparing for train[cat_count_features] 
cat_fea = 
['ps_ind_02_cat','ps_ind_04_cat','ps_ind_05_cat', 'ps_car_01_cat', 'ps_car_02_cat',
 'ps_car_03_cat','ps_car_04_cat','ps_car_05_cat','ps_car_06_cat','ps_car_07_cat',
 'ps_car_08_cat','ps_car_09_cat','ps_car_10_cat', 'ps_car_11_cat']

Example: 
ps_ind_02_cat_count
dictionay of ps_ind_02_cat 
([(1, 1079327), (2, 309747), (3, 70172), (4, 28259), (-1, 523)])

row        count     origial value
595202    1079327       1     
595203     309747       2
595204     309747       2
595205      70172       3
595206    1079327       1

''' 

cat_fea = [ name for name in list(train) if 'cat' in name and 'count' not in name]
features= cat_fea + ['new_ind','new_reg','new_car']

train, test, cat_count_features= Features_Counts(train, test, features)


    

In [8]:
# display(train[['new_ind','new_reg','new_car']].head(5))
display(train[cat_count_features].head(5))

,ps_ind_02_cat_count,ps_ind_04_cat_count,ps_ind_05_cat_count,ps_car_01_cat_count,ps_car_02_cat_count,ps_car_03_cat_count,ps_car_04_cat_count,ps_car_05_cat_count,ps_car_06_cat_count,ps_car_07_cat_count,ps_car_08_cat_count,ps_car_09_cat_count,ps_car_10_cat_count,ps_car_11_cat_count,new_ind_count,new_reg_count,new_car_count
0,309747,620936,1319412,124587,1234979,1028142,1241334,431560,77845,1383070,249663,486510,1475460,18326,6,24,1
1,1079327,866864,1319412,518725,1234979,1028142,1241334,666910,329890,1383070,1238365,883326,1475460,12535,36,38,11
2,28259,620936,1319412,449617,1234979,1028142,1241334,666910,147714,1383070,1238365,883326,1475460,19943,24,13477,40
3,1079327,866864,1319412,449617,1234979,183044,1241334,431560,329890,1383070,1238365,36798,1475460,212989,2784,222,1
4,309747,620936,1319412,518725,1234979,1028142,1241334,666910,147714,1383070,1238365,883326,1475460,26161,258,34,13


## 2.3 Get the feature from feature training ## 

In [6]:
train_fea0, test_fea0 = pickle.load(open("../input/fea0.pk",'rb'), encoding='iso-8859-1')

## 2.4 Statistic features ##

- find the feature of median, mean and standard deviation

In [10]:
#feature aggregation
target_features = ['ps_car_13', 'ps_ind_03', 'ps_reg_03', 'ps_ind_15', 'ps_reg_01', 'ps_ind_01']
group_features = ['ps_car_13', 'ps_ind_03', 'ps_reg_03', 'ps_ind_15', 'ps_reg_01', 'ps_ind_01', 'ps_ind_05_cat']

#return numpy because we need to do np.hstack to merge all statistic feature together, so that it would return np array
train_statis, test_statis =  Statistic_features(train, test, target_features, group_features)

In [11]:
display(train_statis)

array([[  1.57043000e+05,   8.32786207e-01,   2.41530046e-01, ...,
          1.00000000e+00,   7.00000000e+00,   0.00000000e+00],
       [  1.30452000e+05,   8.26528390e-01,   2.35133348e-01, ...,
          1.00000000e+00,   7.00000000e+00,   0.00000000e+00],
       [  6.35510000e+04,   8.13168936e-01,   2.35946815e-01, ...,
          1.00000000e+00,   7.00000000e+00,   0.00000000e+00],
       ..., 
       [  3.58630000e+04,   8.00633360e-01,   2.34463222e-01, ...,
          1.00000000e+00,   7.00000000e+00,   0.00000000e+00],
       [  2.04836000e+05,   8.24270444e-01,   2.28649975e-01, ...,
          1.00000000e+00,   7.00000000e+00,   0.00000000e+00],
       [  9.85280000e+04,   8.24453229e-01,   2.37806003e-01, ...,
          1.00000000e+00,   7.00000000e+00,   0.00000000e+00]])

## 2.5 Combine all feature together, and get ready for training ##

1. merge features into train_list & test_list that would like to dump into NN model
2. training a scaler by sparse that generated by train_list & test_list
4. convert train_list & test_list into X , X_test, which has been scaled

In [12]:
'''
Building a train list including train_num, cat_count_features, statistic feature and infered feature training by XGboost.

train_num: training data without set of cat_calc
cat_count_features: cat_fea + ['new_ind','new_reg','new_car']
train_fea0: feature extraction 
'''


#training data without set of cat_calc
train_num = train[[x for x in list(train) if x in num_features]]
test_num = test[[x for x in list(train) if x in num_features]]

train_list = [train_num.replace([np.inf, -np.inf, np.nan], 0), train[cat_count_features], train_statis, train_fea0 ]
test_list = [test_num.replace([np.inf, -np.inf, np.nan], 0), test[cat_count_features], test_statis,test_fea0] 

In [13]:
'''
X are stacked from 5 features
1. train_num(595212,54): training data without set of cat_calc
2. cat_count_features(595212,17): cat_fea + ['new_ind','new_reg','new_car']
3. feature statis(595212,6) * 36
4. train_fea0(595212, 38): feature extraction

all_data (595212, 235)
'''


X = sparse.hstack(train_list).tocsr()
X_test = sparse.hstack(test_list).tocsr()

all_data = np.vstack([X.toarray(), X_test.toarray()])
scaler = StandardScaler()
scaler.fit(all_data)
X = scaler.transform(X.toarray())
X_test = scaler.transform(X_test.toarray())


## 2.6 Create Cat_feature for NN embeding training ##
 Don't ask why they doing this! they only tell you what is this
 
 - in the feature NN model of the finial testing data, you would need the list, which likes **[[cat_featrue], X]**
 or **[['ps_ind_02_cat', 'ps_ind_04_cat', 'ps_ind_05_cat',.....,'ps_car_11_cat'], X]**
 
 **_This is to process the above testing data. If you could not understand, that is fine, and just look the next steps_**

In [24]:
#preparing for training cat 
train_cat = train[cat_fea]
test_cat = test[cat_fea]

# convert pd to np.array
X_cat = train_cat.values
tem = test_cat.values

# storing the dimension for embedding layer as an input value
max_cat_values = []

for c in cat_fea:
    
    #nomalize the label
    #LabelEncoder: http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html
    
    le = LabelEncoder()
    x = le.fit_transform(pd.concat([train_cat, test_cat])[c])
    train_cat.loc[:,c] = le.transform(train_cat[c])
    test_cat.loc[:,c] = le.transform(test_cat[c])
    max_cat_values.append(np.max(x))

# Build the final testing data
X_TEST_CAT = []
for i in range(tem.shape[1]):
    X_TEST_CAT.append(tem[:, i].reshape(-1, 1))
X_TEST_CAT.append(X_test)


/Users/stevenhu/anaconda3/envs/run/lib/python3.6/site-packages/pandas/core/indexing.py:630: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [57]:
print('cat_fea:', cat_fea)
print('\nmax_cat_values: ',max_cat_values)

cat_fea: ['ps_ind_02_cat', 'ps_ind_04_cat', 'ps_ind_05_cat', 'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_04_cat', 'ps_car_05_cat', 'ps_car_06_cat', 'ps_car_07_cat', 'ps_car_08_cat', 'ps_car_09_cat', 'ps_car_10_cat', 'ps_car_11_cat']

max_cat_values:  [4, 2, 7, 12, 2, 2, 9, 2, 17, 2, 1, 5, 2, 103]


# 3. Training NN Model with Keras # 

1. Build the model
2. training

## 3.1 Build the model ##

### model structure:  ###
<img src="Jupyter_image/NN_layer.png">

In [47]:
def nn_model():
    inputs = []
    flatten_layers = []
    for e, c in enumerate(cat_fea):
        input_c = Input(shape=(1, ), dtype='int32')
        num_c = max_cat_values[e]
        
        # need to add 1, https://keras.io/layers/embeddings/
        # **input_dim: int > 0. Size of the vocabulary, i.e. maximum integer index + 1.**
        embed_c = Embedding(num_c+1,6,input_length=1)(input_c)
        embed_c = Dropout(0.25)(embed_c)
        flatten_c = Flatten()(embed_c)
        inputs.append(input_c)
        flatten_layers.append(flatten_c)
        
    
    input_num = Input(shape=(X.shape[1],), dtype='float32')
    inputs.append(input_num)
    
    #merge X and embedding layer
    flatten_layers.append(input_num)
    flatten = merge(flatten_layers, mode='concat')

    fc1 = Dense(512, kernel_initializer='he_normal')(flatten)
    fc1 = PReLU()(fc1)
    fc1 = BatchNormalization()(fc1)
    fc1 = Dropout(0.75)(fc1)

    fc1 = Dense(64, kernel_initializer='he_normal')(fc1)
    fc1 = PReLU()(fc1)
    fc1 = BatchNormalization()(fc1)
    fc1 = Dropout(0.5)(fc1)

    outputs = Dense(1, kernel_initializer='he_normal', activation='sigmoid')(fc1)

    model = Model(inputs = inputs, outputs = outputs)
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return (model)

## 3.2 Start to Train ##

In [51]:
"""
#validation fold
NFOLDS = 5
kfold = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=218)

I change "test" to "vaild" because I feel it is clear to understand
"""

cv_train = np.zeros(len(train_label))
cv_pred = np.zeros(len(test_id))

#validation fold
NFOLDS = 5
kfold = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=218)

#with different random see make result stable.
num_seeds = 5
begintime = time()
if cv_only:
    for s in range(num_seeds):
        np.random.seed(s)
        for (train_index, valid_index) in kfold.split(X, train_label):
            
            #assign data from training data and labels to validation data; 
            x_train = X[train_index]
            y_train = train_label[train_index]
            x_valid= X[valid_index]
            y_valid = train_label[valid_index]
            
            # assign X_cat to validation data; 
            x_train_cat = X_cat[train_index]
            x_valid_cat = X_cat[valid_index]

            #Package data for training, the package(list) is  [[cat_featrues], x_train] 
            # or [ ['ps_ind_02_cat', 'ps_ind_04_cat', 'ps_ind_05_cat',.....,'ps_car_11_cat'] ,x_train]
            
            x_train_cat_list, x_valid_cat_list = [], []
            for i in range(x_train_cat.shape[1]):
                x_train_cat_list.append(x_train_cat[:, i].reshape(-1, 1))
                x_valid_cat_list.append(x_valid_cat[:, i].reshape(-1, 1))

            x_train_cat_list.append(x_train)
            x_valid_cat_list.append(x_valid)
            
            #load model
            model = nn_model()
            
            def get_rank(x):
                return pd.Series(x).rank(pct=True).values
            #fit model. Note: Change epochs to make prediction accuracy
            model.fit(x_train_cat_list, y_train, epochs=10, batch_size=512, verbose=2, validation_data=[x_valid_cat_list, y_valid])
            
            #record prediction with validation data
            cv_train[valid_index] += get_rank(model.predict(x=x_valid_cat_list, batch_size=512, verbose=0)[:, 0])
            print('local fold Gini: ',Gini(train_label[valid_index], cv_train[valid_index]))
            
            #recode prediction with testing data
            cv_pred += get_rank(model.predict(x=X_TEST_CAT, batch_size=512, verbose=0)[:, 0])
             
            
        
        print("seed {0}: Gini {1}".format(s,Gini(train_label, cv_train / (1. * (s + 1)))))
        print("Total training time: ",str(datetime.timedelta(seconds=time() - begintime)))
    if save_cv:
        
        #divid (NFOLDS * num_seeds) to get average of probablity 
        pd.DataFrame({'id': test_id, 'target': get_rank(cv_pred * 1./ (NFOLDS * num_seeds))}).to_csv('../model/keras5_pred.csv', index=False)
        pd.DataFrame({'id': train_id, 'target': get_rank(cv_train * 1. / num_seeds)}).to_csv('../model/keras5_cv.csv', index=False)



/Users/stevenhu/anaconda3/envs/run/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/Users/stevenhu/anaconda3/envs/run/lib/python3.6/site-packages/keras/legacy/layers.py:465: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


Train on 297606 samples, validate on 297606 samples
Epoch 1/1
 - 27s - loss: 0.3061 - val_loss: 0.1639
local fold Gini:  0.209322322663
Train on 297606 samples, validate on 297606 samples
Epoch 1/1
 - 28s - loss: 0.3087 - val_loss: 0.1645
local fold Gini:  0.201585256464
seed 0: Gini 0.20545379936910999
Total training time:  0:01:55.265348
